<a href="https://colab.research.google.com/github/Htets-Corner/SYNTHBUSTER_RAISE-1k/blob/main/syn32_real_Mobilenetv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 0: Mount Drive and Import Libraries

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import core libraries
import os
import numpy as np
import matplotlib.pyplot as plt

# Torch and torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# Utilities
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


Mounted at /content/drive


In [3]:
!ls drive/MyDrive/

'Colab Notebooks'   RAISE_1k.csv.zip   SYNTHBUSTER_32.zip
 RAISE		    synthbuster        synthbuster.zip


In [9]:
folder_path = "drive/MyDrive/RAISE/PNG"
# Count only .png files
png_count = sum(1 for f in os.listdir(folder_path) if f.lower().endswith(".png"))

print(f"Number of PNG files in '{folder_path}': {png_count}")

Number of PNG files in 'drive/MyDrive/RAISE/PNG': 999


In [4]:
# Step 1: Load Datasets and Split into Train/Test

import kagglehub

# Download Synthbuster (JPEG resized version) from Kaggle
synth_path = kagglehub.dataset_download("devpatel484/synthbuster-32")
print("Path to Synthbuster dataset:", synth_path)

100%|██████████| 9.04M/9.04M [00:00<00:00, 55.5MB/s]

Extracting files...


Path to Synthbuster dataset: /root/.cache/kagglehub/datasets/devpatel484/synthbuster-32/versions/1


In [7]:
# Paths
real_path = "/content/drive/MyDrive/RAISE/PNG"   # Real images
ai_path   = synth_path   # AI images (JPEGs)

# Define transforms (resize, normalize, augment for train)
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # MobileNetV2 expects 224x224
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Combine both datasets under one structure using ImageFolder
# Let's first create parent folders with 'real' and 'ai' subdirs

!mkdir -p /content/dataset/real
!mkdir -p /content/dataset/ai

# Symlink instead of copying (saves space)
!ln -s "{real_path}"/* /content/dataset/real/
!ln -s "{ai_path}"/* /content/dataset/ai/

# Now create dataset with ImageFolder
full_dataset = datasets.ImageFolder(
    root="/content/dataset",
    transform=transform_train
)

# Train/Test split (80/20)
train_size = int(0.8 * len(full_dataset))
test_size  = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(
    full_dataset, [train_size, test_size]
)

# Apply correct transforms
train_dataset.dataset.transform = transform_train
test_dataset.dataset.transform  = transform_test

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"Total images: {len(full_dataset)}")
print(f"Train: {len(train_dataset)}, Test: {len(test_dataset)}")
print(f"Classes: {full_dataset.classes}")

Total images: 9999
Train: 7999, Test: 2000
Classes: ['ai', 'real']


In [ ]:
!ls dataset/ai/resized_data_Synthbuster/Synthbuster_Dataset/
#!ls dataset/real